Introduction:

The first professional baseball league was created in 1871, and the sport has been at the forefront of American entertainment and culture for over 150 years.  Although the framework of the game has remained unchanged, shifts in overall game strategy, team management, and player skillsets have evolved.

Defined in 1980 by the godfather of baseball analytics Bill James as "the search for objective knowledge about baseball", sabremetrics has for decades impacted how team owners and front offices evaulate players and construct teams.  With the advent of high speed cameras and other technological advancements that enable the tracking of pitch movement and the launch angles of batted balls an entire new regime of data, and tons of it, are now available.  The influx has turned baseball managers into part time data analysts and the increasing reliance on analytics to evaulate players has caused some of this terminology and thought process to bleed into discourse of the common fan, something not even the biggest baseball nerd could have predicted 15 years ago.

As a result, several publically available sources of baseball data now exist on the web, including the well known website 'Fangraphs' which provides hundreds of team and individual player metrics as well as analytically minded articles.  Using this and other sources I thought it would be interesting to see how well the available data could help us predict the winner of baseball's World Series.

The following is a project I undertook to follow that initiave.

The first step of the project is to define the objective(s) of the analysis.  For this project I intend to develop a model that, using a priori data, is capable of predicting who will win the 2018 World Series.  To accomplish this I will use yearly full-season batting statistics for each team from the year 2000 to the current year, 2018.  Because we already know who has won the World Series from 2000 to 2017, we can use machine learning to develop a model to predict the outcome of this year's championship.  The batting statistics will be obtained as .csv files from Fangraphs.  The information on which team won the World Series each year is obtained from another online source called the Lahman Database.  Since we will be using data from two different sources together it will be important to first load the data, and re-organize it to resolve formatting differeces between the two databases.

Our Fangraphs data is in the form of .csv files with a separate file for each year where the year is appended onto the end of the file name.  The first task is to load this data into a single dataframe.

In [1]:
import pandas as pd
import numpy as np
import glob
   
file_names = glob.glob('FanGraphs Leaderboard_*.csv')
                               
data_frames = [pd.read_csv(f) for f in file_names]
        
for i, sheet in enumerate(data_frames):
  sheet['year'] = int(file_names[i][-8:-4]) 
            
fg = pd.concat(data_frames)

Let's make sure our concatenation worked properly.  Each file is from a separate year, so concatenating them we would expect that each of the years from 2000 to 2018 are represented

In [2]:
fg.drop_duplicates('year')['year']

0    2000
0    2001
0    2002
0    2003
0    2004
0    2005
0    2006
0    2007
0    2008
0    2009
0    2010
0    2011
0    2012
0    2013
0    2014
0    2015
0    2016
0    2017
0    2018
Name: year, dtype: int64

OK, now let's have a quick look at the data

In [3]:
fg.head()

,Team,Age,G,AB,PA,H,1B,2B,3B,HR,...,wXX/C (pi),O-Swing% (pi),Z-Swing% (pi),Swing% (pi),O-Contact% (pi),Z-Contact% (pi),Contact% (pi),Zone% (pi),Pace (pi),year
0,Indians,29,1930,5683,6512,1639,1078,310,30,221,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2000
1,Astros,29,2303,5570,6444,1547,973,289,36,249,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2000
2,Giants,29,2123,5519,6418,1535,961,304,44,226,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2000
3,White Sox,27,1791,5646,6410,1615,1041,325,33,216,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2000
4,Athletics,28,2066,5560,6432,1501,958,281,23,239,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2000


Already we can see that there are 286 features and a lot of missing data, particularly from the earlier years when these data points were not being logged. With a bit of information on some of these statistics I think it makes sense to pair off some of the pitch f/x data, which tracks the movement of pitches.  This is an evaluation of team offense first and foremost and pitch f/x data is difficult for a lay-person like myself to interpret anyway.  In the dataset, pitch f/x data is labeled with 'pfx' or 'pi' which gives us a convenient way to remove it.

In [4]:
searchfor = '|'.join(['pi','pfx'])
mask = fg.columns.str.contains(searchfor)
fg_pared = fg.loc[:,~mask]
fg_pared.shape

(570, 126)

So we've effectively reduced the dataset by more than 50%, but we still have over 100 features which should be enough for a good model.  Now, let's import the Lahman database data which gives us information on which team won the World Series each year.  The data goes back to the beginning of baseball, but we only need the years that match the data we've already acquired.

In [5]:
teams = pd.read_csv('Teams.csv')
m = (fg.year.min() <= teams['yearID']) & (teams['yearID'] <= fg.year.max())
teams_pared = teams.loc[m,:]

The data we're interested in are those that pair the team with whether they won the WS that year.

In [6]:
teams_pared[['yearID', 'teamID', 'name', 'WSWin']]

,yearID,teamID,name,WSWin
2325,2000,ANA,Anaheim Angels,N
2326,2000,ARI,Arizona Diamondbacks,N
2327,2000,ATL,Atlanta Braves,N
2328,2000,BAL,Baltimore Orioles,N
2329,2000,BOS,Boston Red Sox,N
2330,2000,CHA,Chicago White Sox,N
2331,2000,CHN,Chicago Cubs,N
2332,2000,CIN,Cincinnati Reds,N
2333,2000,CLE,Cleveland Indians,N
2334,2000,COL,Colorado Rockies,N


Merging this dataset with the Fangraphs dataset on team name seems to be the most straight forward way to approach this, but unfortunatley the Lahman dataset includes the city along with the team name.  Before merging the datasets we'll need to reconcile this and can do so by removing the city name from the Lahman name data column.  Almost all team names are the last word of the entire team name phrase (e.g. New York Yankees = Yankees), so we can create a command that generates a new name column only using the last word of each team name phrase.

In [7]:
new_names = teams_pared['name'].str.split().str[-1:].str.join("")
teams_pared['new_name'] = new_names

C:\Users\Stefano\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [8]:
teams_pared['new_name']

2325          Angels
2326    Diamondbacks
2327          Braves
2328         Orioles
2329             Sox
2330             Sox
2331            Cubs
2332            Reds
2333         Indians
2334         Rockies
2335          Tigers
2336         Marlins
2337          Astros
2338          Royals
2339         Dodgers
2340         Brewers
2341           Twins
2342           Expos
2343         Yankees
2344            Mets
2345       Athletics
2346        Phillies
2347         Pirates
2348          Padres
2349        Mariners
2350          Giants
2351       Cardinals
2352            Rays
2353         Rangers
2354            Jays
            ...     
2835    Diamondbacks
2836          Braves
2837         Orioles
2838             Sox
2839             Sox
2840            Cubs
2841            Reds
2842         Indians
2843         Rockies
2844          Tigers
2845          Astros
2846          Royals
2847         Anaheim
2848         Dodgers
2849         Marlins
2850         Brewers
2851         

This works for most teams, except those that have names longer than one word (e.g. Blue Jays).  We'll need to write some extra code to handle these situations individually

In [9]:
RS = (teams_pared['name'] == 'Boston Red Sox')
BJ = (teams_pared['name'] == 'Toronto Blue Jays')
WS = (teams_pared['name'] == 'Chicago White Sox')
LAA = (teams_pared['name'] == 'Los Angeles Angels of Anaheim')
RAY = (teams_pared['name'] == 'Tampa Bay Devil Rays')
teams_pared['new_name'].where(~RS, "Red Sox", inplace = True)
teams_pared['new_name'].where(~BJ, "Blue Jays", inplace = True)
teams_pared['new_name'].where(~WS, "White Sox", inplace = True)
teams_pared['new_name'].where(~LAA, "Angels", inplace = True)
teams_pared['new_name'].where(~RAY, "Devil Rays", inplace = True)

C:\Users\Stefano\Anaconda3\lib\site-packages\pandas\core\generic.py:5984: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [10]:
print(teams_pared['new_name'][teams_pared['new_name'] == 'White Sox'])
print('\nNumber of times "Sox" is incorrectly used in dataframe: %d' % (teams_pared['new_name'] == 'Sox').sum())

2330    White Sox
2360    White Sox
2390    White Sox
2420    White Sox
2450    White Sox
2479    White Sox
2509    White Sox
2539    White Sox
2569    White Sox
2599    White Sox
2629    White Sox
2659    White Sox
2689    White Sox
2719    White Sox
2749    White Sox
2779    White Sox
2809    White Sox
2839    White Sox
Name: new_name, dtype: object

Number of times "Sox" is incorrectly used in dataframe: 0


Here as an example we can see that the White Sox team name has been corrected from 'Sox' to 'White Sox'.  Next, we can pare off all the unecessary data.  From this data set we really only need the year, the team name, and whether that team won the World Series

In [11]:
teams_pared = teams_pared[['yearID', 'WSWin', 'new_name']]

Now we can merge the two sources of data on the team name and year

In [12]:
total_data = pd.merge(fg_pared, teams_pared, left_on = ['Team', 'year'], 
                      right_on = ['new_name', 'yearID'], how = 'outer')
total_data.set_index(total_data['year'], inplace = True)

Looking through the data one realizes that some of the statistics are represented as strings, which will be a problem.  Before performing some final touch-ups with the data it will be important to recognize these columns of data and either delete them if they are not useful or convert them to float type data.

In [13]:
df_obj = total_data.select_dtypes(['object'])
df_obj.dropna().head()

,Team,BB%,K%,LD%,GB%,FB%,IFFB%,HR/FB,IFH%,BUH%,...,Age Rng,Pull%,Cent%,Oppo%,Soft%,Med%,Hard%,TTO%,WSWin,new_name
year,,,,,,,,,,,,,,,,,,,,,
2004,Yankees,10.5 %,15.4 %,18.5 %,44.9 %,36.6 %,9.7 %,14.5 %,7.6 %,20.3 %,...,20 - 39,45.8 %,28.5 %,25.7 %,15.5 %,56.3 %,28.2 %,29.8 %,N,Yankees
2004,Indians,9.4 %,15.6 %,19.9 %,43.5 %,36.7 %,11.4 %,10.7 %,5.8 %,21.1 %,...,21 - 37,45.0 %,28.1 %,27.0 %,12.9 %,58.2 %,28.8 %,27.9 %,N,Indians
2004,Rockies,9.0 %,18.6 %,19.8 %,45.1 %,35.1 %,10.1 %,13.2 %,6.1 %,18.7 %,...,23 - 41,41.2 %,28.8 %,30.0 %,14.3 %,58.1 %,27.7 %,30.8 %,N,Rockies
2004,Cardinals,8.7 %,17.2 %,21.1 %,43.0 %,35.8 %,10.1 %,13.3 %,6.8 %,20.5 %,...,21 - 37,41.3 %,29.6 %,29.1 %,13.9 %,59.0 %,27.2 %,29.3 %,N,Cardinals
2004,Phillies,10.0 %,17.5 %,21.4 %,42.6 %,36.0 %,9.9 %,13.2 %,5.8 %,18.3 %,...,21 - 39,43.1 %,29.9 %,27.0 %,14.1 %,55.6 %,30.3 %,30.9 %,N,Phillies


The main problem is that data represented as percentages in our dataframe are strings which include the '%' sign.  A deeper dive also shows that there are other problematic symbols such as '$' and parentheses '(' and ')'.  We need to introduce some code to scrub off those strings and convert the data into numerical format.

In [14]:
for line in df_obj.columns:
    if any(total_data[line].fillna('').str.contains('\\$')):
        total_data[line] = total_data[line].str.strip('(').str.strip(")").str.strip("$").astype(float)
    elif any(total_data[line].fillna('').str.contains('\\%')):
        total_data[line] = total_data[line].str.strip(' %').astype(float)

In [15]:
df_obj = total_data.select_dtypes(['object'])
df_obj.columns

Index(['Team', 'Age Rng', 'WSWin', 'new_name'], dtype='object')

We've successfully transformed all % data from string to float data types.  The only numerical string data we haven't dealt with is the 'Age Rng' statistic, which is a string that gives the age range of the players on a team.  This won't be helpful in our analysis, so we can remove it.  The WSWin column is a 'Y' or 'N' string denoting whether a specific team won the World Series in a given year.  We'll be using that information to build our classifier model, so let's do a bit of feature engineering and convert 'Y' to 1 and 'N' to 0.

In [16]:
total_data.drop(labels = ['new_name', 'Age Rng', 'year', 'yearID'], 
                axis = 1, inplace = True)
total_data['WSWin'].replace({'Y': 1, 'N': 0}, inplace = True)

The goal of the project is to be able to predict who will win the 2018 World Series.  With that in mind let's remove the 2018 regular season statistics from the current data set and assign it to its own dataframe.  Organizing the data this way will allow us to create a model using data from 2000 to 2017 and then use the finished product to predict the World Series on the 2018 regular season data.

In [17]:
data_2018 = total_data[total_data.index == 2018]
total_data.drop(index = 2018, inplace = True)

Our data is now ready for the next stage in the project, which will be dealing with missing data and/or highly correlated variables.